# Go High Level

## Search Conversations

https://marketplace.gohighlevel.com/docs/ghl/conversations/search-conversation/index.html

In [23]:
import requests
import os
import json

from dotenv import load_dotenv

load_dotenv()

TOKEN = os.environ.get("GHL_TOKEN")

location_id = "JMKW7uSiXL63XD0a2duU"
url = f"https://services.leadconnectorhq.com/conversations/search?locationId={location_id}"

payload = {}
headers = {
    "Accept": "application/json",
    "Version": "2021-04-15",
    "Authorization": f"Bearer {TOKEN}",
}

response = requests.request("GET", url, headers=headers, data=payload)
response_json = json.loads(response.text)
response_json["total"]

613

## Get messages by conversation id

https://marketplace.gohighlevel.com/docs/ghl/conversations/get-messages/index.html

In [24]:
conv_id = "hEKzEHN89kZuQkHePyir"

url = f"https://services.leadconnectorhq.com/conversations/{conv_id}/messages"

payload = {}
headers = {
    "Accept": "application/json",
    "Version": "2021-04-15",
    "Authorization": f"Bearer {TOKEN}",
}

response = requests.request("GET", url, headers=headers, data=payload)

response_json = json.loads(response.text)

response_json["messages"]

{'lastMessageId': 'at1GmNIrlgfScOB2d8O2',
 'nextPage': False,
 'messages': [{'id': 'at1GmNIrlgfScOB2d8O2',
   'type': 3,
   'locationId': 'JMKW7uSiXL63XD0a2duU',
   'contactId': 'BGEAyBCqByriTjRAkhAr',
   'conversationId': 'hEKzEHN89kZuQkHePyir',
   'dateAdded': '2025-08-10T15:02:06.822Z',
   'dateUpdated': '2025-08-10T15:02:06.822Z',
   'meta': {'email': {'messageIds': ['MJ1hQRHRuXPq300EAAYS']}},
   'source': 'bulk_actions',
   'messageType': 'TYPE_EMAIL'}]}

## Get message by message id

https://marketplace.gohighlevel.com/docs/ghl/conversations/get-message

In [26]:
import requests

message_id = "Yh1VoppdfE45AnmyHtek"
url = f"https://services.leadconnectorhq.com/conversations/messages/{message_id}"

payload = {}
headers = {
    "Accept": "application/json",
    "Version": "2021-04-15",
    "Authorization": f"Bearer {TOKEN}",
}

response = requests.request("GET", url, headers=headers, data=payload)

response_json = json.loads(response.text)
response_json


{'message': {'id': 'Yh1VoppdfE45AnmyHtek',
  'type': 3,
  'locationId': 'JMKW7uSiXL63XD0a2duU',
  'body': '<html>\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=us-ascii">\r\n<style type="text/css" style="display:none;"> P {margin-top:0;margin-bottom:0;} </style>\r\n</head>\r\n<body dir="ltr">\r\n<div style="font-family: Aptos, Aptos_EmbeddedFont, Aptos_MSFontService, Calibri, Helvetica, sans-serif; font-size: 12pt; color: rgb(0, 0, 0);" class="elementToProof">\r\nManoj,</div>\r\n<div style="font-family: Aptos, Aptos_EmbeddedFont, Aptos_MSFontService, Calibri, Helvetica, sans-serif; font-size: 12pt; color: rgb(0, 0, 0);" class="elementToProof">\r\nIt was good to meet and I appreciate you reaching out.<br>\r\nI have attached I document that I hope helps. If it is too much, let me know and I can send something simpler. My thought is that you can use this document with cntrl+f to search zip codes.</div>\r\n<div style="font-family: Aptos, Aptos_EmbeddedFont, Aptos

## Email Cleaning

https://chatgpt.com/share/e/6898b4fe-c8ac-8001-8802-e7abbd2afd14

In [4]:
from bs4 import BeautifulSoup, Comment
import re
import unicodedata

# --- Heuristics tuned for EN-only threads ---
REPLY_MARKERS = [
    r"^On .+ wrote:\s*$",
    r"^On .+ at .+ wrote:\s*$",
    r"^On .+,\s*.+ wrote:\s*$",
    r"^-----Original Message-----$",
    r"^Begin forwarded message:$",
    r"^Forwarded message:$",
    r"^From:\s?.+$",
    r"^To:\s?.+$",
    r"^Sent:\s?.+$",
    r"^Subject:\s?.+$",
    r"^Reply above this line$",
]
SIG_MARKERS = [
    r"^\s*--\s*$",  # standard sig delimiter
    r"^Thanks,?$",
    r"^Thank you,?$",
    r"^Best( regards)?,?$",
    r"^Kind regards,?$",
    r"^Regards,?$",
    r"^Sincerely,?$",
    r"^Cheers,?$",
    r"^Sent from my iPhone",
    r"^Sent from my Android",
]

FOOTER_NOISE_RE = re.compile(
    r"(unsubscribe|manage preferences|update your preferences|privacy|terms|confidentiality|do not reply|view this email in your browser)",
    re.I,
)
ZERO_WIDTH_RE = re.compile(r"\u200b|\u200c|\u200d|\ufeff")
MULTISPACE_RE = re.compile(r"[ \t]+")
EXTRA_NEWLINES_RE = re.compile(r"\n{3,}")


def _is_hidden(el) -> bool:
    style = (el.get("style") or "").lower()
    if any(k in style for k in ("display:none", "visibility:hidden", "opacity:0")):
        return True
    if el.get("aria-hidden") == "true":
        return True
    w = (el.get("width") or "").lower()
    h = (el.get("height") or "").lower()
    if w in {"1", "1px"} and h in {"1", "1px"}:
        return True
    return False


def _strip_reply_and_signature(lines):
    reply_rx = [re.compile(p, re.I) for p in REPLY_MARKERS]
    sig_rx = [re.compile(p, re.I) for p in SIG_MARKERS]
    kept = []
    for line in lines:
        if line.startswith(">"):  # quoted block
            break
        if any(rx.search(line) for rx in reply_rx):
            break
        if any(rx.search(line) for rx in sig_rx):
            break
        kept.append(line)
    return kept


def _replace_links(soup: BeautifulSoup, keep_links: bool):
    for a in soup.find_all("a"):
        text = (a.get_text(strip=True) or "").strip()
        href = (a.get("href") or "").strip()
        if not href or not keep_links or href.lower().startswith("javascript:"):
            a.replace_with(text)
        else:
            a.replace_with(f"{text} ({href})" if text else href)


def _prepare_lists(soup: BeautifulSoup):
    # Turn list items into bullet lines to preserve structure
    for li in soup.find_all("li"):
        li.insert_before("\n- ")
        li.insert_after("\n")


def clean_email_html(html: str, *, keep_links: bool = True) -> str:
    """
    Convert messy email HTML to clean plaintext.
      - remove scripts/styles/hidden nodes/trackers
      - drop common footer boilerplate (unsubscribe/legal)
      - preserve bullets and (optionally) links
      - strip quoted reply history and signatures
    """
    # 1) Parse
    soup = BeautifulSoup(html or "", "lxml")

    # Remove comments early
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        c.extract()

    # Remove noisy tags entirely
    for tag in soup(["script", "style", "noscript", "svg", "form", "iframe", "head"]):
        tag.decompose()

    # Remove hidden/tracking elements
    for el in list(soup.find_all(True)):
        if _is_hidden(el):
            el.decompose()

    # Drop footer/legal/unsubscribe blocks
    for node in list(soup.find_all(string=FOOTER_NOISE_RE)):
        block = node.find_parent(["footer", "table", "div", "section", "p"])
        if block:
            block.decompose()

    # 2) Light structural normalization
    for br in soup.find_all("br"):
        br.replace_with("\n")
    _prepare_lists(soup)
    _replace_links(soup, keep_links=keep_links)

    # Add newlines around blocks so words don’t jam together
    for blk in soup.find_all(
        [
            "p",
            "div",
            "section",
            "tr",
            "ul",
            "ol",
            "table",
            "h1",
            "h2",
            "h3",
            "h4",
            "h5",
            "h6",
        ]
    ):
        blk.insert_before("\n")
        blk.insert_after("\n")

    # 3) Extract text
    text = soup.get_text(separator=" ", strip=True)

    # 4) Normalize whitespace and unicode
    text = unicodedata.normalize("NFKC", text)
    text = ZERO_WIDTH_RE.sub("", text)
    text = MULTISPACE_RE.sub(" ", text)
    text = EXTRA_NEWLINES_RE.sub("\n\n", text)

    # 5) Strip quoted replies and signatures
    lines = [ln.rstrip() for ln in text.splitlines()]
    lines = _strip_reply_and_signature(lines)

    out = "\n".join(lines).strip()
    return EXTRA_NEWLINES_RE.sub("\n\n", out)

In [48]:
sample = response_json["message"]["body"]

clean_email_html(sample)

'Hi Jon, Nice seeing you at the convention. Is it possible to mail me available territories and the hot markets to present to my clients please. Thank you Best, Manoj Soans Franchise Consultant Franchises Global 1224 N Broadway, Santa Ana, CA 92701 p:(310)999-1670 e: (mailto:e%3Amanojsoans@franchisesglobal.com) manojsoans@franchisesglobal.com (mailto:manojsoans@franchisesglobal.com) w: www.franchisesglobal.com (http://www.franchisesglobal.com)'